# ĐỒ ÁN: LẬP TRÌNH CHO KHOA HỌC DỮ LIỆU
## Chủ đề: **Giá phòng trọ ở khu vực Hồ Chí Minh**
- Môn học: Lập Trình Cho Khoa Học Dữ Liệu
- Nhóm: 6

## THU THẬP DỮ LIỆU

### 1. Cài đặt thư viện, import

In [ ]:
!pip install scrapy
!pip install spider3
!pip install unidecode

In [1]:
import scrapy
import re
import pandas as pd
import json

### 2. Tạo project

In [ ]:
!scrapy startproject phongtro123

In [2]:
cd phongtro123\phongtro123

C:\Users\ParaboY\Desktop\LTKHDL_Project_Phongtro\phongtro123\phongtro123


#### 2.1. Thu thập URL của các bài post cho thuê nhà ở Hồ Chí Minh

Code trong file `spiders/collect_url.py`

In [ ]:
import scrapy
import re

class Phongtro123Spider(scrapy.Spider):
    name = 'phongtro123_get_url'
    start_urls = ['https://phongtro123.com/tinh-thanh/ho-chi-minh?page=1']
    
    def __init__(self):
        self.page_count = 1
        self.end_page =0

    def parse(self, response):
        for item in response.css('div.post-meta'):
            link=item.css('a::attr(href)').get()
            if link!=None:
                yield {
                    'link':'https://phongtro123.com'+ link
                }
            #print(link)
        
        if self.page_count == 1:
            end_page = response.css('#left-col > ul > li:nth-child(6) > a').attrib['href']
            end_page=int(re.findall('[0-9]+',end_page)[-1])
            self.end_page=end_page
        
        self.page_count=self.page_count+1
        if self.page_count<=self.end_page:
            next_page='https://phongtro123.com/tinh-thanh/ho-chi-minh?page='+ str(self.page_count)
            yield scrapy.Request(response.urljoin(next_page), callback=self.parse)
            
        print('Number of page:'+str(end_page))


#### 2.2. Thu thập các thông tin trên từng bài post

Các trường thông tin thu thập trên mỗi bài post:
- **Id**: Mã tin quảng cáo cho thuê trọ.
- **Title**: Tiêu đề tin quảng cáo cho thuê trọ .
- **Address**: Địa chỉ cho thuê trọ.
- **Price**: Mức giá cho thuê trọ.
- **Acreage**: Diện tích phòng trọ.
- **Content**: Thông tin mô tả phòng trọ.
- **Type_post**: Loại tin rao quảng cáo.
- **Tenant**: Đối tượng có thể thuê trọ.
- **Posting_time**: Thời gian đăng thông báo cho thuê trọ.
- **End_time**: Thời gian kết thúc cho thuê trọ.
- **Contact**: Tên người có thể liên hệ nếu khách muốn thuê trọ.
- **Phone_number**: Số điện thoại người liên hệ.
- **Zalo**: Số Zalo người liên hệ.
- **Link**: Link của bài post

Code trong file `spiders/collect_info.py`

In [ ]:
# -*- coding: utf-8 -*-

import scrapy
import json
import re
import datetime

import unidecode

def remove_accent(text):
    if text==None:
        return ""
    return unidecode.unidecode(text)

def listToString(s):
 
    # initialize an empty string
    str1 = ""
 
    # traverse in the string
    for ele in s:
        str1 += ele + ' || '
 
    # return string
    return str1

class collect_player_info(scrapy.Spider):
    name='phongtro123_get_info'
  
    def __init__(self):
        try:
            with open('post_url_temp.json') as f:
                self.urls = json.load(f)
                self.url_count = 1
                self.url=''
        except IOError:
            print("File not found")

    def start_requests(self):
        urls = self.urls[0]['link']
        self.url=urls
        yield scrapy.Request(url=urls, callback=self.parse)
  
    def parse(self, response):
        
        info={}        

        
        title=response.xpath('//*[@id="left-col"]/article/header/h1/a/text()').extract_first()
        info['title']=remove_accent(title)
        
        address=response.xpath('//*[@id="left-col"]/article/header/address/text()').extract_first()
        info['address']=remove_accent(address)
        
        price=response.xpath('//*[@id="left-col"]/article/header/div[2]/div[1]/span/text()').extract_first()
        info['price']=remove_accent(price)
        
        acreage=response.xpath('//*[@id="left-col"]/article/header/div[2]/div[2]/span/text()').extract_first()
        info['acreage']=remove_accent(acreage)
        
        
        content=' || '.join(response.css('#left-col > article > section.section.post-main-content > div.section-content p::text').getall())
        info['content']=remove_accent(content)
             
        
        table = response.xpath('//*[@id="left-col"]/article/section[2]/div[2]/table/tr')
        s=remove_accent(table.get())
        
        match=re.search(r'Ma tin:</td><td>#([^<]+)',s)
        if match:
            info['id']=match.group(1)
        else:
            info['id']=None
            
        match=re.search(r'Loai tin rao:</td><td>([^<]+)',s)
        if match:
            info['type_post']=match.group(1)
        else:
            info['type_post']=None
            
        match=re.search(r'Doi tuong thue:</td><td>([^<]+)',s)
        if match:
            info['tenant']=match.group(1)
        else:
            info['tenant']=None
            
        match=re.search(r'Ngay dang:</td><td><time title="([^"]+)">',s)
        if match:
            info['posting_time']=match.group(1)
        else:
            info['posting_time']=None
            
        match=re.search(r'Ngay het han:</td><td><time title="([^"]+)">',s)
        if match:
            info['end_time']=match.group(1)
        else:
            info['end_time']=None
            
        match=re.search(r'Lien he:</td><td> ([^<]+)',s)
        if match:
            info['contact']=match.group(1)
        else:
            info['contact']=None
            
        match=re.search(r'Dien thoai:</td><td> ([^<]+)',s)
        if match:
            info['phone_number']=match.group(1)
        else:
            info['phone_number']=None
            
        match=re.search(r'Zalo</td><td> ([^<]+)',s)
        if match:
            info['zalo']=match.group(1)
        else:
            info['zalo']=None
        
        info['link']=self.url
        #info['type_post']=re.search(r'Loai tin rao:</td><td>([^<]+)',s).group(1)
        #info['tenant']=re.search(r'Doi tuong thue:</td><td>([^<]+)',s).group(1)
        
        #info['posting_time']=re.search(r'Ngay dang:</td><td><time title="([^"]+)">', s).group(1)
        #info['end_time']=re.search(r'Ngay het han:</td><td><time title="([^"]+)">', s).group(1)
        #info['contact']=re.search(r'Lien he:</td><td> ([^<]+)', s).group(1)
        #info['phone_number']=re.search(r'Dien thoai:</td><td> ([^<]+)', s).group(1)     
        #info['zalo']=re.search(r'Zalo</td><td> ([^<]+)', s).group(1)  
        #print(info)
        
        yield info
      
        if self.url_count < len(self.urls):
            next_page_url = self.urls[self.url_count]['link']
            self.url=next_page_url
            self.url_count += 1
            yield scrapy.Request(response.urljoin(next_page_url), callback=self.parse) 

### 3. Scraping data

#### 3.1. Thu thập link các bài post ở Hồ Chí Minh

Thu thập dữ liệu và ghi vào file `dataset/post_url.json`

In [ ]:
!scrapy crawl phongtro123_get_url -o dataset/post_url.json

Loại bỏ trùng lặp:

In [ ]:
import pandas as pd
df = pd.read_json(r'dataset/post_url.json')
df.head()

In [ ]:
df.shape

In [ ]:
df=df.drop_duplicates()

In [ ]:
df.shape

Ghi lại vào file.

In [ ]:
#df.to_json(r'dataset/post_url_temp.json',orient='records')
json_str = json.dumps(df.to_dict('records'), ensure_ascii=False)
with open(r'dataset/post_url.json', 'w') as f:
    f.write(json_str)

#### 3.2. Thu thập thông tin của từng bài post

Tách thành nhiều file url nhỏ để dễ xử lý hơn khi các link thu thập bị lỗi (có thể do người đăng bài post đó xóa bài, lỗi khi thu thập url, lỗi do truy cập vào trang web quá nhiều, mạng lag,...)

Ở đây em tách thành 29 file url

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[0:1000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

Thu thập thông tin của từng bài post

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_01.json

**Tương tự**

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[1000:2000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_02.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[2000:3000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_03.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[3000:4000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_04.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[4000:5000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_05.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[5000:6000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_06.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[6000:7000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_07.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[7000:8000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_08.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[8000:9000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_09.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[9000:10000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_10.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[10000:11000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_11.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[11000:12000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_12.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[12000:13000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_13.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[13000:14000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_14.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[14000:15000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_15.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[15000:16000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_16.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[16000:17000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_17.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[17000:18000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_18.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[18000:19000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_19.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[19000:20000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_20.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[20000:25000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_21.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[25000:30000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_22.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[30000:35000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_23.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[35000:40000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_24.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[40000:45000,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_25.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[45000:,:]
print(df1.shape)

json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_26.json

**Xử lý các file url chứa url bị lỗi**

Thu thập lại từ những phần bị lỗi

- 03

- 13

- 23

Ta sẽ cào lại những trên khoảng url bị lỗi và chỉ cào những url chưa được cào.

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[2000:3000,:]
df2= pd.read_json('dataset/post_info_03.json')

- Tìm những url chưa được cào

In [ ]:
# Merge the two DataFrames and specify the 'indicator' parameter
df_merged = df1.merge(df2, on='link', how='outer', indicator=True)

# Select rows where the '_merge' column is 'left_only'
df_filtered = df_merged[df_merged['_merge'] == 'left_only']

# Drop  some column
#df_filtered.drop(columns=['_merge'], inplace=True)
df1=pd.DataFrame(df_filtered['link'])

# Print the resulting DataFrame
print(df1)

- Ghi đè lại vào file url

In [ ]:
json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

- Cào dữ liệu

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_27.json

**Tương tự**

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[12000:13000,:]
df2= pd.read_json('dataset/post_info_13.json')

In [ ]:
# Merge the two DataFrames and specify the 'indicator' parameter
df_merged = df1.merge(df2, on='link', how='outer', indicator=True)

# Select rows where the '_merge' column is 'left_only'
df_filtered = df_merged[df_merged['_merge'] == 'left_only']

# Drop  some column
#df_filtered.drop(columns=['_merge'], inplace=True)
df1=pd.DataFrame(df_filtered['link'])

# Print the resulting DataFrame
print(df1)

In [ ]:
json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_28.json

In [ ]:
df1 = pd.read_json('dataset/post_url.json')
df1 = df1.iloc[30000:35000,:]
df2= pd.read_json('dataset/post_info_23.json')

In [ ]:
# Merge the two DataFrames and specify the 'indicator' parameter
df_merged = df1.merge(df2, on='link', how='outer', indicator=True)

# Select rows where the '_merge' column is 'left_only'
df_filtered = df_merged[df_merged['_merge'] == 'left_only']

# Drop  some column
#df_filtered.drop(columns=['_merge'], inplace=True)
df1=pd.DataFrame(df_filtered['link'])


df1 = df1.loc[df1['link'] != 'https://phongtro123.com/tinh-thanh/ho-chi-minh/hinh-thachi-minh/phong-tro-nha-nguyen-can-cao-cap-moi-100-mat-tien-pham-the-hien.html']
df1 = df1.loc[df1['link'] != "https://phongtrot-phong-3-5tr-wc-trong-phong-moi-xay-cua-so-lon-gio-tu-do-q11.html"]
# Print the resulting DataFrame
print(df1)

In [ ]:
json_str = json.dumps(df1.to_dict('records'), ensure_ascii=False)
with open(r'post_url_temp.json', 'w') as f:
    f.write(json_str)

In [ ]:
!scrapy crawl phongtro123_get_info -o dataset/post_info_29.json

**Merge toàn bộ file post_info (từ 01 - 29) thành 1 file post_info.json**

In [ ]:
import json

# Read the contents of the JSON files into strings
json_strs = [open(f'dataset/post_info_{i:02d}.json', 'r').read() for i in range(1, 30)]

# Deserialize the JSON strings into Python objects
objects = [json.loads(json_str) for json_str in json_strs]

# Merge the objects into a single object
# Merge the objects into a single object
data_merged = []
for obj in objects:
    for key in obj:
        data_merged.append(key)

# Serialize the merged object into a JSON string
json_str = json.dumps(data_merged,indent=2)

# Write the JSON string to a new file
with open('dataset/post_info.json', 'w') as f:
    f.write(json_str)

### 4. Kết quả

In [3]:
df = pd.read_json('dataset/post_url.json')
print("Số lượng link đã thu thập: ",df.shape[0])

Số lượng link đã thu thập:  49121


In [4]:
df = pd.read_json('dataset/post_info.json')
print("Số lượng thông tin giá thuê đã thu thập: ",df.shape[0])

Số lượng thông tin giá thuê đã thu thập:  49119


Số lượng thông tin đã thu thập có chênh lệch so số lượng link thu thập là do thời gian để thu thập toàn bộ dữ liệu tương đối lâu, và trong thời gian đó đã có thể có bài post bị xóa, hoặc link bị lỗi khi thực hiện bước thu thập url.

In [5]:
df1 = pd.read_json('dataset/post_url.json')
df2= pd.read_json('dataset/post_info.json')

df_merged = df1.merge(df2, on='link', how='outer', indicator=True)

In [6]:
print('Các link bị lỗi: ' ,list(df_merged[df_merged['_merge'] == 'left_only']['link']))

Các link bị lỗi:  ['https://phongtro123.com/tinh-thanh/ho-chi-minh/hinh-thachi-minh/phong-tro-nha-nguyen-can-cao-cap-moi-100-mat-tien-pham-the-hien.html', 'https://phongtrot-phong-3-5tr-wc-trong-phong-moi-xay-cua-so-lon-gio-tu-do-q11.html']
